In [ ]:
import functools
import itertools
import os
from typing import Dict, List, Optional

import pandas as pd
import textdistance

from fm_matcher.utils.models import Attribute, Parameters, Relation

In [ ]:
benchmark = pd.read_csv("benchmark/ground_truth.csv")
for side in ("source", "target"):
    benchmark[[f"{side}_schema", f"{side}_relation", f"{side}_attribute"]] = benchmark[side].str.split(".", expand=True)
    benchmark[side] = benchmark[side].str.lower()
benchmark["benchmark"] = True

relation_combinations = benchmark[["source_relation", "target_relation"]].drop_duplicates().values

In [ ]:
@functools.cache
def get_description(schema: str, table: str, attribute: Optional[str] = None) -> str:
    if attribute:
        filename = f"{schema}_{table}_{attribute}.txt"
    else:
        filename = f"{schema}_table_{table}.txt"
    filename_filter = lambda f: f.lower() == filename.lower()
    filename = next(filter(filename_filter, os.listdir("schema_documentations")))
    with open(os.path.join("schema_documentations", filename), "r") as desc_file:
        description = desc_file.read()
    return description


@functools.cache
def get_attributes(schema: str, table: str) -> List[str]:
    table_filter = lambda f: f.lower().startswith(f"{schema}_{table}_") and f.endswith(".txt")
    extract_attr_name = lambda f: f[len(f"{schema}_{table}_"):-len(".txt")]
    return [
        extract_attr_name(attr_file)
        for attr_file in filter(table_filter, os.listdir("schema_documentations/"))
    ]


@functools.cache
def get_relation(schema: str, table: str) -> Relation:
    schema, table = schema.lower(), table.lower()
    description = get_description(schema, table)
    attributes = [
        Attribute(
            name=attr_name.capitalize(),
            description=get_description(schema, table, attr_name),
        ) for attr_name in get_attributes(schema, table)
    ]
    return Relation(
        name=table.capitalize(),
        description=description,
        attributes=attributes,
    )


parameters = [
    Parameters(
        source_relation=get_relation("mimic", source),
        target_relation=get_relation("omop", target),
    ) for source, target in relation_combinations
]

In [ ]:
baseline_data = []
for param in parameters:
    for source, target in itertools.product(param.source_relation.attributes, param.target_relation.attributes):
        baseline_data.append({
            "source": f"mimic.{param.source_relation.name}.{source.name}".lower(),
            "source_relation": param.source_relation.name,
            "source_attribute": source.name.lower(),
            "target": f"omop.{param.target_relation.name}.{target.name}".lower(),
            "target_relation": param.target_relation.name,
            "target_attribute": target.name.lower(),
            "jaro": textdistance.jaro.normalized_similarity(source.name.lower(), target.name.lower()),
        })
baseline_df = pd.DataFrame(baseline_data)
baseline_df = baseline_df.merge(benchmark[["source", "target", "benchmark"]], on=["source", "target"], how="left").copy()
baseline_df["benchmark"] = baseline_df["benchmark"].fillna(False)
baseline_df.to_csv("results/baseline_results.csv", index=False)
baseline_df